<a href="https://colab.research.google.com/github/laurahsisson/smoker-status/blob/main/MultiLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Prediction of Smoker Status using Bio-Signals
[From Kaggle Playground Series](https://www.kaggle.com/competitions/playground-series-s3e24/overview)

In [12]:
import pandas as pd
import torch
import sklearn
import sklearn.model_selection

In [13]:
%pip install torchmetrics
import torchmetrics

In [14]:
%pip install optuna
import optuna

In [15]:
from google.colab import drive
import os

drive.mount('/content/drive')
input_dir = "/content/drive/MyDrive/Data/Playground/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
train = pd.read_csv(os.path.join(input_dir, "train.csv"))
evaluate = pd.read_csv(os.path.join(input_dir, "test.csv"))
sample_submission = pd.read_csv(os.path.join(input_dir, "sample_submission.csv"))

In [17]:
print(f"Training data of size = {train.shape}")
print(f"Test data of size = {evaluate.shape}")

Training data of size = (159256, 24)
Test data of size = (106171, 23)


In [18]:
x_columns = train.columns[train.columns != "smoking"]
y_column = train.columns[train.columns == "smoking"]

print("Input data for id = 0")
display(train.iloc[0][x_columns])
print()
print("Target value for id = 0")
display(train.iloc[0][y_column])

Input data for id = 0


id                       0.0
age                     55.0
height(cm)             165.0
weight(kg)              60.0
waist(cm)               81.0
eyesight(left)           0.5
eyesight(right)          0.6
hearing(left)            1.0
hearing(right)           1.0
systolic               135.0
relaxation              87.0
fasting blood sugar     94.0
Cholesterol            172.0
triglyceride           300.0
HDL                     40.0
LDL                     75.0
hemoglobin              16.5
Urine protein            1.0
serum creatinine         1.0
AST                     22.0
ALT                     25.0
Gtp                     27.0
dental caries            0.0
Name: 0, dtype: float64


Target value for id = 0


smoking    1.0
Name: 0, dtype: float64

In [19]:
# Min max normalzie values
x_train = train[x_columns]
x_train_normalized = (x_train-x_train.min())/(x_train.max()-x_train.min())
x_train_normalized.sample(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
97683,0.613375,0.230769,0.545455,0.35,0.434211,0.112245,0.112245,0.0,0.0,0.279412,...,0.145119,0.275591,0.077999,0.701863,0.0,0.071429,0.029793,0.009269,0.045135,0.0
7789,0.048909,0.153846,0.636364,0.40,0.368421,0.112245,0.142857,0.0,0.0,0.242647,...,0.125330,0.433071,0.074771,0.720497,0.0,0.102041,0.019430,0.006522,0.036108,0.0
78386,0.492204,0.461538,0.272727,0.30,0.421053,0.091837,0.091837,1.0,1.0,0.455882,...,0.310026,0.377953,0.050565,0.521739,0.0,0.051020,0.027202,0.006866,0.024072,0.0
2895,0.018178,0.307692,0.727273,0.55,0.618421,0.112245,0.112245,0.0,0.0,0.169118,...,0.304749,0.346457,0.074233,0.633540,0.0,0.112245,0.023316,0.007209,0.077232,1.0
27838,0.174801,0.307692,0.454545,0.25,0.221053,0.142857,0.112245,0.0,0.0,0.227941,...,0.027704,0.535433,0.084454,0.608696,0.0,0.071429,0.036269,0.006522,0.016048,0.0


In [20]:
eval_normalized = (evaluate-x_train.min())/(x_train.max()-x_train.min())
eval_normalized.sample(5)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
14046,1.088204,0.307692,0.454545,0.2,0.230263,0.142857,0.142857,0.0,0.0,0.301471,...,0.098945,0.503937,0.055944,0.478261,0.0,0.061224,0.063472,0.015105,0.016048,0.0
42189,1.264921,0.230769,0.545455,0.4,0.468421,0.040816,0.061224,0.0,0.0,0.455882,...,0.183377,0.220472,0.074233,0.633540,0.0,0.081633,0.015544,0.009612,0.037111,0.0
32100,1.201570,0.384615,0.727273,0.6,0.607895,0.061224,0.051020,0.0,0.0,0.352941,...,0.193931,0.251969,0.048951,0.670807,0.0,0.102041,0.011658,0.004463,0.027081,0.0
44532,1.279633,0.307692,0.545455,0.5,0.486842,0.142857,0.142857,0.0,0.0,0.205882,...,0.205805,0.259843,0.060247,0.677019,0.0,0.051020,0.022021,0.010985,0.024072,0.0
50442,1.316744,0.615385,0.454545,0.4,0.605263,0.061224,0.051020,0.0,0.0,0.536765,...,0.182058,0.236220,0.076385,0.639752,0.0,0.091837,0.025907,0.007209,0.025075,0.0


In [21]:
train_x_tensor = torch.FloatTensor(x_train_normalized.to_numpy())
train_y_tensor = torch.FloatTensor(train[y_column].to_numpy())
eval_x_tensor = torch.FloatTensor(eval_normalized.to_numpy())

print(f"train_x_tensor = {train_x_tensor.shape}",f"train_y_tensor = {train_y_tensor.shape}",f"eval_x_tensor = {eval_x_tensor.shape}",sep="\n")
print(train_x_tensor.min(),train_x_tensor.max())
print(eval_x_tensor.min(),eval_x_tensor.max())

train_x_tensor = torch.Size([159256, 23])
train_y_tensor = torch.Size([159256, 1])
eval_x_tensor = torch.Size([106171, 23])
tensor(0.) tensor(1.)
tensor(-0.0449) tensor(1.6667)


In [22]:
def get_device():
    device = "cpu"
    if torch.cuda.is_available():
      device = "cuda:0"
    return device

class SmokingDataset(torch.utils.data.Dataset):
    def __init__(self,data,target=None):
        self.data = data.to(get_device())

        # Null coalesce the target to an empty of the expected shape.
        target = target if torch.is_tensor(target) else torch.empty((len(data),1))
        self.target = target.to(get_device())

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]

        return {'data': x, 'target': y}

    def __len__(self):
        return len(self.data)

bsz = 512
tr_x, ts_x, tr_y, ts_y = sklearn.model_selection.train_test_split(train_x_tensor, train_y_tensor)
print(f"tr_x = {tr_x.shape}",f"tr_y = {tr_y.shape}",sep=",\t")
print(f"ts_x = {ts_x.shape}",f"ts_y = {ts_y.shape}",sep=",\t")

train_loader = torch.utils.data.DataLoader(SmokingDataset(tr_x,tr_y), batch_size=bsz)
test_loader = torch.utils.data.DataLoader(SmokingDataset(ts_x,ts_y), batch_size=bsz)

example_batch = next(iter(train_loader))
example_batch["data"].shape, example_batch["target"].shape

tr_x = torch.Size([119442, 23]),	tr_y = torch.Size([119442, 1])
ts_x = torch.Size([39814, 23]),	ts_y = torch.Size([39814, 1])


(torch.Size([512, 23]), torch.Size([512, 1]))

In [26]:
def make_sequential(input_dim, hidden_dim, output_dim, dropout):
  return torch.nn.Sequential(torch.nn.Linear(input_dim, hidden_dim), torch.nn.ReLU(), torch.nn.Dropout(p=dropout), torch.nn.Linear(hidden_dim, output_dim))

def make_deep_sequential(input_dim, depth, hidden_dim, output_dim, dropout):
  all_dims = [input_dim] + depth * [hidden_dim] + [output_dim]
  layers = []

  for i in range(len(all_dims)-1):
    layers.append(torch.nn.Sequential(torch.nn.Linear(all_dims[i], all_dims[i+1]),
                                      torch.nn.ReLU(),
                                      torch.nn.Dropout(p=dropout)))

  # No dropout or non-linearity on the last layer.
  layers[-1] = torch.nn.Sequential(torch.nn.Linear(all_dims[-2], all_dims[-1]))

  return torch.nn.Sequential(*layers)

def do_epoch(models,train_loader,test_loader):
  model = models["model"]
  opt = models["opt"]
  loss_fn = models["loss_fn"]
  auroc_fn = models["auroc_fn"]

  for batch in train_loader:
    opt.zero_grad()
    fwd = model(batch["data"])
    loss = loss_fn(fwd,batch["target"])
    loss.backward()
    opt.step()


  with torch.no_grad():
    ts_pred = []
    ts_target = []
    for batch in test_loader:
      ts_pred.append(model(batch["data"]))
      ts_target.append(batch["target"])

  return auroc_fn(torch.cat(ts_pred),torch.cat(ts_target)).cpu().numpy()

def get_config(trial):
  return {
    "lr": trial.suggest_float("lr", 1e-10,1e-1,log=True),
    "dropout": trial.suggest_float("dropout", 0,1),
    # "depth": trial.suggest_int("depth",0,3),
    "hidden_dim": trial.suggest_int("hidden_dim",2,512,log=True),
    "epochs": trial.suggest_int("epochs",1,100),
  }

def make_models(config):
  model = make_sequential(len(x_columns),config["hidden_dim"],1,config["dropout"])
  model.to(get_device())
  return {
    "loss_fn": torch.nn.BCEWithLogitsLoss(),
    "model": model,
    "opt": torch.optim.Adam(model.parameters(),lr=config["lr"]),
    "auroc_fn": torchmetrics.AUROC(task="binary"),
  }

def train_model(trial):
  config = get_config(trial)

  models = make_models(config)

  for i in range(config["epochs"]):
    loss = do_epoch(models,train_loader,test_loader)

    trial.report(loss, i)
    if i % 10 == 0:
      print(i,loss)
    if trial.should_prune():
        raise optuna.TrialPruned()

  return loss

In [24]:
print(f"Num GPUS = {torch.cuda.device_count()}")

Num GPUS = 1


In [ ]:
from functools import partial

num_trials = 100

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(train_model, n_trials=num_trials)

print("DONE")
print("CONFIG:", study.best_trial.params)
print("METRICS:", study.best_trial.values)

[I 2023-10-25 20:38:18,905] A new study created in memory with name: no-name-1bae3176-b0c3-4f1c-a965-04fee60eceec


0 0.5658778
10 0.5685642
20 0.5704104


[I 2023-10-25 20:39:12,137] Trial 0 finished with value: 0.5661314725875854 and parameters: {'lr': 3.923911556205562e-07, 'dropout': 0.5358582060026729, 'hidden_dim': 3, 'epochs': 28}. Best is trial 0 with value: 0.5661314725875854.


0 0.6721304
10 0.7044803
20 0.71189564
30 0.721102
40 0.7316072
50 0.7403537
60 0.7472389
70 0.7525548
80 0.75822043
90 0.7628261


[I 2023-10-25 20:41:29,472] Trial 1 finished with value: 0.7641903162002563 and parameters: {'lr': 3.8075155689721195e-06, 'dropout': 0.023589568154011142, 'hidden_dim': 84, 'epochs': 94}. Best is trial 1 with value: 0.7641903162002563.


...

[I 2023-10-25 20:48:34,176] Trial 40 pruned. 


0 0.5000061


[I 2023-10-25 20:48:37,041] Trial 41 pruned. 


0 0.6494583


[I 2023-10-25 20:48:39,679] Trial 42 pruned. 


In [ ]:
# Need to remember to make submission predictions with logits
all_loader = torch.utils.data.DataLoader(SmokingDataset(train_x_tensor,train_y_tensor), batch_size=bsz)
eval_loader = torch.utils.data.DataLoader(SmokingDataset(eval_x_tensor), batch_size=bsz)

models = make_models(study.best_trial.params)
for epoch in range(study.best_trial.params["epochs"]):
  loss = do_epoch(models,all_loader,all_loader)
  print(loss)

eval_pred = []
with torch.no_grad():
  for batch in eval_loader:
    eval_pred.append(models["model"](batch["data"]))
eval_prob = torch.nn.functional.sigmoid(torch.cat(eval_pred)).squeeze().cpu()
print(f"eval_prob = {eval_prob.shape} w/ final loss = {loss}")

submission = pd.DataFrame({"id" : evaluate.id, "smoking" : eval_prob}).set_index("id")
display(submission)
submission.to_csv(os.path.join(input_dir, "Submissions", "first.csv"))
print("Made submission!")